In [4]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import time, datetime

data = pd.read_csv('../../data/gold_clean.csv')
data = data.rename(columns = {'ts': 'unixtime'})
data['ts'] = pd.to_datetime(data.unixtime)
data = data[(data.ts >= '2018-01-10') & (data.ts <= '2018-01-11')]

data = data.reset_index(drop=True)
print(data.head())

data['rolling_mean'] = data.close.rolling(window = 100).mean()

for index, row in data.iterrows():
    if (row['rolling_mean'] < row['close']):
        print(row['rolling_mean'])
        break


data['rolling_std'] = data.close.rolling(window = 100).std()
for index, row in data.iterrows():
    if not np.isnan(row['rolling_std']):
        print(row['rolling_std'])
        break

              unixtime    open    high     low   close  vol  \
0  1515542400000000000  1314.8  1315.0  1314.7  1314.9  247   
1  1515542460000000000  1314.9  1315.2  1314.8  1315.0  395   
2  1515542520000000000  1315.0  1315.1  1315.0  1315.1   67   
3  1515542580000000000  1315.1  1315.1  1314.8  1314.9   39   
4  1515542640000000000  1314.8  1314.9  1314.8  1314.9   22   

                   ts  
0 2018-01-10 00:00:00  
1 2018-01-10 00:01:00  
2 2018-01-10 00:02:00  
3 2018-01-10 00:03:00  
4 2018-01-10 00:04:00  
1311.306
1.044861887349508


In [5]:
#https://medium.datadriveninvestor.com/how-to-implement-bollinger-bands-in-python-1106b90da8d1
data['upper_band'] = data['rolling_mean'] + 3 * data['rolling_std']
data['lower_band'] = data['rolling_mean'] - 3 * data['rolling_std']

print(data[data['close'] > data['upper_band']].close.count())
print(data[data['close'] < data['lower_band']].close.count())

15
6


In [6]:
def mm(value):
    return (max(value) + min(value))*0.5

data['rolling_mm'] = data.close.rolling(window = 50).apply(mm, raw = False)

print(data['rolling_mm'][-1:])

1380    1318.85
Name: rolling_mm, dtype: float64


In [7]:
data['rolling_w5m'] = data.close.rolling(window = 500).mean()
data['rolling_ewm'] = data.close.ewm(span = 100).mean()

v1 = np.sign(data.rolling_w5m - data.rolling_ewm)
v1 = np.diff(v1)
v1 = np.where((v1 != 0) & (~np.isnan(v1)))[0]
print(v1)

[ 604 1092]
